In this project we are going to analyze data from the classifieds section of the german eBay website, the main focus of this  project is to practice Data cleansing.

**Importing Data**

In [609]:
import pandas as pd
import numpy as np


In [610]:
autos=pd.read_csv('autos.csv',encoding='Windows-1252' )

**Data Checking** 

This part focus on check whether the data was successfully imported and the first glance at what kind of data and possible issues

In [611]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The first observation is that some columns label is in Camel case(different words together by an uppercase)  the columns price and odometer should be numeric but are text.

**Fixing columns labels** 

from camelcase to snakecase

In [612]:
# a copy of the data Frame
autoscopy=autos.copy()


In [613]:
def camel_to_snake(x):#check is a string has an uppercase and replace it by an underline and lowercase 
    if x.isupper():
        x=x.lower()
        x='_'+x
    return x

new_columns=[]
for l in autoscopy.columns:# a loop over the columns label string  
    word=[]
    for a in l:#check each character of the columns label to detect an uppercase
        s=camel_to_snake(a)
        word.append(s)
    word=''.join(word)
    new_columns.append(word)
autoscopy.columns=new_columns

In [614]:
autoscopy.columns


Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'year_of_registration', 'gearbox', 'power_p_s', 'model',
       'odometer', 'month_of_registration', 'fuel_type', 'brand',
       'not_repaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

Now the columns label problem is solved.

**Removing columns**

In [615]:
describe=autoscopy.describe(include='all')
describe[0:2]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,year_of_registration,gearbox,power_p_s,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.0,47320,50000.0,47242,50000,50000.0,45518,50000,40171,50000,50000.0,50000.0,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481


There are some columns with a small number of unique values that me delete to clear our dataset.

This dataset is not large and I could manually check column by column, but to prepare for a large one I will automate the process using a filter of less than 1% of unique values in comparison to count.

In [616]:
describe_unique_percent=describe.iloc[1]/describe.iloc[0]
candidates=list(describe_unique_percent.index[(describe_unique_percent<0.01)|(describe_unique_percent.isnull())])
candidates# values with less than 1% or Nan of unique values

['seller',
 'offer_type',
 'abtest',
 'vehicle_type',
 'year_of_registration',
 'gearbox',
 'power_p_s',
 'model',
 'odometer',
 'month_of_registration',
 'fuel_type',
 'brand',
 'not_repaired_damage',
 'date_created',
 'nr_of_pictures',
 'postal_code']

The values above are our candidates to be removed we are going to apply the method value_counts() to see the blocks of repeated values. any value related to date will be kept because even if they are repeated they still can provide useful information. 

In [617]:
candidates.remove('year_of_registration')
candidates.remove('month_of_registration')
candidates.remove('date_created')

Making a list of columns  to be removed if it has less 4 blocks of unique values

In [618]:
remove_candidates=[]
for i in candidates:
    if len(autoscopy[i].value_counts())<4:
        remove_candidates.append(i)

In [619]:
remove_candidates

['seller',
 'offer_type',
 'abtest',
 'gearbox',
 'not_repaired_damage',
 'nr_of_pictures']

These are the value to be removed from our dataset

In [620]:
autoscopy=autoscopy.drop(remove_candidates,axis=1)

**Checking Data type**

In [621]:
autoscopy.info()
autoscopy.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   date_crawled           50000 non-null  object
 1   name                   50000 non-null  object
 2   price                  50000 non-null  object
 3   vehicle_type           44905 non-null  object
 4   year_of_registration   50000 non-null  int64 
 5   power_p_s              50000 non-null  int64 
 6   model                  47242 non-null  object
 7   odometer               50000 non-null  object
 8   month_of_registration  50000 non-null  int64 
 9   fuel_type              45518 non-null  object
 10  brand                  50000 non-null  object
 11  date_created           50000 non-null  object
 12  postal_code            50000 non-null  int64 
 13  last_seen              50000 non-null  object
dtypes: int64(4), object(10)
memory usage: 5.3+ MB


,date_crawled,name,price,vehicle_type,year_of_registration,power_p_s,model,odometer,month_of_registration,fuel_type,brand,date_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",bus,2004,158,andere,"150,000km",3,lpg,peugeot,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",limousine,1997,286,7er,"150,000km",6,benzin,bmw,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",limousine,2009,102,golf,"70,000km",7,benzin,volkswagen,2016-03-26 00:00:00,35394,2016-04-06 20:15:37


date_crawled, price, odometer,date_created and last seen are numeric data stored as text

let's start fixing price and odometer

In [622]:
autoscopy.price.head()

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object

The problem with price is that we have '$' and ',' 

In [623]:
#removing '$' and ','
autoscopy.price=autoscopy.price.str.replace('$','')
autoscopy.price=autoscopy.price.str.replace(',','')
autoscopy.price=autoscopy.price.astype(float)


The problem with odometer is 'km' and ','

In [624]:
#Removing 'km' and ','
autoscopy.odometer=autoscopy.odometer.str.replace('km','')
autoscopy.odometer=autoscopy.odometer.str.replace(',','')
autoscopy.odometer=autoscopy.odometer.astype(float)

In [625]:
autoscopy=autoscopy.rename(columns={'odometer':'odometer_km'})

**Analyzing range values.**

Price

In [626]:
print(autoscopy.price.describe())
print(autoscopy.price.unique().shape)


count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64
(2357,)


In [627]:
print(autoscopy.price.value_counts(normalize=True).head().sort_index(ascending=True))


0.0       0.02842
500.0     0.01562
1200.0    0.01278
1500.0    0.01468
2500.0    0.01286
Name: price, dtype: float64


Starting from the bottom, some values are to cheap to be true, even cars that are for free.

In [628]:
#let's remove all the price values of 0
autoscopy=autoscopy[autoscopy.price>0]
print(autoscopy.price.describe())
print(autoscopy.price.value_counts(normalize=True).head().sort_index(ascending=True))

count    4.857900e+04
mean     1.012788e+04
std      4.880873e+05
min      1.000000e+00
25%      1.200000e+03
50%      3.000000e+03
75%      7.490000e+03
max      1.000000e+08
Name: price, dtype: float64
500.0     0.016077
1000.0    0.013154
1200.0    0.013154
1500.0    0.015109
2500.0    0.013236
Name: price, dtype: float64


we still believe that a car would be too cheap for a value less than 500 euros, but because it is good to keep a Margin of safety let's see the number or cars bellow 100  

In [629]:
autoscopy[autoscopy.price<100].shape

(341, 14)

Because there are only 341 values, we can safely remove these values from our analysis.


For the top most expensive cars, the intutition says that is too expensive to be realistics because the most expensive cars in the world are around 10million,but once your car is out loot there is a deprecation of 7%, so let's see our maximum value as 9.0million

In [630]:
autoscopy=autoscopy[autoscopy.price.between(100, 9*10**6, inclusive=True)]


In [631]:
autoscopy.price.describe()

count    4.823000e+04
mean     6.125042e+03
std      2.286169e+04
min      1.000000e+02
25%      1.250000e+03
50%      3.000000e+03
75%      7.499000e+03
max      3.890000e+06
Name: price, dtype: float64

Let's check if there is any redflag in the odometer_km values.

In [632]:
autoscopy.odometer_km.describe()

count     48230.000000
mean     125916.545718
std       39546.680447
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [633]:
print(autoscopy.odometer_km.value_counts().head().sort_index(ascending=False))

150000.0    31215
125000.0     5038
100000.0     2101
90000.0      1733
80000.0      1412
Name: odometer_km, dtype: int64


There is not problem with values of odometer_km, which has a range from 5000 to 150000 km, values perfectly aceptables.The majority of the vehicles are more than 150000km 

The dates values are saved as string, lets convert ir to a data format because it has hours let's first only on the first 10 strings

In [634]:
autoscopy.date_crawled=autoscopy.date_crawled.str[:10]
autoscopy.date_crawled=pd.to_datetime(autoscopy['date_crawled'])

In [635]:
autoscopy.date_crawled.describe()

count                   48230
unique                     34
top       2016-04-03 00:00:00
freq                     1862
first     2016-03-05 00:00:00
last      2016-04-07 00:00:00
Name: date_crawled, dtype: object

The date range is from 2016-03-05 to 2016-04-07   

In [636]:
autoscopy.date_created=autoscopy.date_created.str[:10]
autoscopy.date_created=pd.to_datetime(autoscopy['date_created'])

In [637]:
autoscopy.date_created.describe()

count                   48230
unique                     76
top       2016-04-03 00:00:00
freq                     1874
first     2015-06-11 00:00:00
last      2016-04-07 00:00:00
Name: date_created, dtype: object

the date range is from 2015-06-11 to 2016-04-07

In [638]:
autoscopy.last_seen.str[:10].value_counts(normalize=True,dropna=False).sort_index()
autoscopy.last_seen=pd.to_datetime(autoscopy['last_seen'])

In [639]:
autoscopy.last_seen.describe()

count                   48230
unique                  38237
top       2016-04-07 06:17:27
freq                        8
first     2016-03-05 14:45:46
last      2016-04-07 14:58:50
Name: last_seen, dtype: object

the range is from 2016-03-05 to 2016-04-07

In [640]:
autoscopy.year_of_registration.describe()

count    48230.000000
mean      2004.729795
std         87.892058
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: year_of_registration, dtype: float64

There is a min value of 1000 and max of 9999 since these values are impossible. This database is from  2016 and a car cannot be produced after 2016 and cars  mass production started in 1908 by  Ford Motor Company

In [641]:
autoscopy=autoscopy[autoscopy.year_of_registration.between(1908,2017)]
autoscopy.year_of_registration.describe()

count    47741.000000
mean      2003.346139
std          7.409503
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2017.000000
Name: year_of_registration, dtype: float64

most of our analysis vehicles were produced in the 2000s which matches my intuition.

**Data analysis**

In this part, we are going to analyze the top 5 brands and find out if the most populars ones are because of their price or other aspects e.g marketing or quality 


In [642]:
toplist=autoscopy.brand.value_counts().sort_values(ascending=False)
toplist=toplist[:10]
toplist

volkswagen       10159
bmw               5202
opel              5156
mercedes_benz     4586
audi              4118
ford              3332
renault           2273
peugeot           1413
fiat              1232
seat               888
Name: brand, dtype: int64

My first observation is that all top 5 brands are german, which would suggest some nationalism because this dataset is from eBay in germany.

In [643]:
#market mean
market_mean=autoscopy.price.mean()
#calculates the mean for each brand
top_brand={}
brands=toplist.index
for x in brands:
    select=autoscopy[autoscopy['brand']==x]
    selected=select.price.mean()
    top_brand[x]=selected
top_brand=pd.DataFrame.from_dict(top_brand, orient='index')    

In [644]:
top_brand

,0
volkswagen,5580.219510
bmw,8567.470204
opel,2983.042281
mercedes_benz,8572.807239
audi,9286.365226
ford,4060.090036
renault,2462.856137
peugeot,3090.864827
fiat,2813.182630
seat,4388.873874


Now let's see check the average of each brand to the market brand a number bellow 1 means that the brand average is smaller than market average

In [647]:
brand_to_market=top_brand/market_mean
brand_to_market

,0
volkswagen,0.907992
bmw,1.394065
opel,0.485389
mercedes_benz,1.394934
audi,1.511041
ford,0.660642
renault,0.400746
peugeot,0.502934
fiat,0.457750
seat,0.714141


we conclude that most of the popular brands have a price below the market average, the cars that are popular and have a price above the averages suggest that people who are selling expect people who are not considering being affordable as the main reason for a purchasing.